In [1]:
import numpy as np
import os
import tensorflow as tf
import urllib
import tensorflowjs as tfjs
import json
import keras

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  4


## Downloading model from GitHub

In [4]:
urllib.request.urlretrieve(
    "https://github.com/ur-whitelab/peptide-dashboard/raw/master/models/hemo-rnn/keras_model/model_weights.h5",
    "model_weights.h5",
)
urllib.request.urlretrieve(
    "https://github.com/ur-whitelab/peptide-dashboard/raw/master/models/hemo-rnn/keras_model/model.json",
    "model.json",
)
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = keras.models.model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("model_weights.h5")
print("Loaded model from disk.")

2022-04-06 19:24:55.914701: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-06 19:24:58.329019: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38417 MB memory:  -> device: 0, name: A100-SXM4-40GB, pci bus id: 0000:01:00.0, compute capability: 8.0
2022-04-06 19:24:58.330759: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 38417 MB memory:  -> device: 1, name: A100-SXM4-40GB, pci bus id: 0000:41:00.0, compute capability: 8.0
2022-04-06 19:24:58.331869: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 

Loaded model from disk.


## Making predictions

In [5]:
vectorized_seq = np.array([[10, 1, 18, 1, 11, 16, 1, 13, 12, 10, 11, 3, 8, 7, 14]])

# function counts_aa obtains amino acid counts frequency vector
def counts_aa(vec):
    counts =  tf.histogram_fixed_width(vec, [0, 20], nbins=21)[1:]
    return counts /tf.reduce_sum(counts)


# inputs.shape[-1] needs to be 190, so we pad zeros to the end
vectorized_seq = np.concatenate([vectorized_seq, np.zeros((vectorized_seq.shape[0], 190-vectorized_seq.shape[-1]))], axis=-1)
vectorized_seq_fr = counts_aa(vectorized_seq)[tf.newaxis,...]
y_predict = model.predict([vectorized_seq, vectorized_seq_fr])
y_predict = np.array([1 if x > 0.5 else 0 for x in y_predict])
print(y_predict)

2022-04-06 19:25:03.886861: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8101


[1]


2022-04-06 19:25:06.025193: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
